Clone airockchip repository

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
!cd yolov5
!pip install -r requirements.txt  # install

Train (clone dataset folder in yolov5 folder before this step)

In [ ]:
!python train.py --img 256 --epochs 100 --data dataset/data.yaml --weights yolov5n.pt

Export to onnx

In [ ]:
!python export.py   --weights runs/train/exp4/weights/best.pt --img 256  --include onnx  --rknpu

Install rknn-toolkit-lite

Export in rknn

In [5]:
import sys
from rknn.api import RKNN

DATASET_PATH = 'dataset/dataset.txt'
DEFAULT_RKNN_PATH = '/media/arina/3054491A0696DA9F/work/yolov5_rockchip/yolov5/runs/train/exp9/weights/best.onnx'
DEFAULT_QUANT = True

def parse_arg():
    if len(sys.argv) < 3:
        print("Usage: python3 {} onnx_model_path [platform] [dtype(optional)] [output_rknn_path(optional)]".format(sys.argv[0]))
        print("       platform choose from [rk3562, rk3566, rk3568, rk3576, rk3588, rv1103, rv1106, rv1126b, rv1109, rv1126, rk1808]")
        print("       dtype choose from [i8, fp] for [rk3562, rk3566, rk3568, rk3576, rk3588, rv1103, rv1106, rv1126b]")
        print("       dtype choose from [u8, fp] for [rv1109, rv1126, rk1808]")
        exit(1)

    model_path = sys.argv[1]
    platform = sys.argv[2]

    do_quant = DEFAULT_QUANT
    model_type = 'i8'
    if len(sys.argv) > 3:
        model_type = sys.argv[3]
        if model_type not in ['i8', 'u8', 'fp']:
            print("ERROR: Invalid model type: {}".format(model_type))
            exit(1)
        elif model_type in ['i8', 'u8']:
            do_quant = True
        else:
            do_quant = False

    if len(sys.argv) > 4:
        output_path = sys.argv[4]
    else:
        output_path = DEFAULT_RKNN_PATH

    return model_path, platform, do_quant, output_path

if __name__ == '__main__':
    # model_path, platform, do_quant, output_path = parse_arg()

    model_path = '/media/arina/3054491A0696DA9F/work/yolov5_rockchip/yolov5/runs/train/exp9/weights/best.onnx'
    platform = 'rv1103' 
    do_quant = True
    output_path = 'yolov5n_256_rockchip.rknn'

        # Create RKNN object
    rknn = RKNN(verbose=False)

    # Pre-process config
    print('--> Config model')
    rknn.config(mean_values=[[0, 0, 0]], std_values=[
                    [255, 255, 255]], target_platform=platform)
    print('done')

    # Load model
    print('--> Loading model')
    ret = rknn.load_onnx(model=model_path)
    if ret != 0:
        print('Load model failed!')
        exit(ret)
    print('done')

    # Build model
    print('--> Building model')
    ret = rknn.build(do_quantization=do_quant, dataset=DATASET_PATH)
    if ret != 0:
        print('Build model failed!')
        exit(ret)
    print('done')

    # Export rknn model
    print('--> Export rknn model')
    ret = rknn.export_rknn(output_path)
    if ret != 0:
        print('Export rknn model failed!')
        exit(ret)
    print('done')

    # Release
    rknn.release()





I rknn-toolkit2 version: 2.3.2


--> Config model
done
--> Loading model


I Loading : 100%|██████████████████████████████████████████████| 121/121 [00:00<00:00, 43634.32it/s]


done
--> Building model


I OpFusing 2 : 100%|████████████████████████████████████████████| 100/100 [00:00<00:00, 2777.57it/s]


I Quantizating : 100%|███████████████████████████████████████████| 149/149 [00:00<00:00, 422.94it/s]
W build: The default input dtype of 'images' is changed from 'float32' to 'int8' in rknn model for performance!
                       Please take care of this change when deploy rknn model with Runtime API!
W build: The default output dtype of 'output0' is changed from 'float32' to 'int8' in rknn model for performance!
                      Please take care of this change when deploy rknn model with Runtime API!
W build: The default output dtype of '286' is changed from 'float32' to 'int8' in rknn model for performance!
                      Please take care of this change when deploy rknn model with Runtime API!
W build: The default output dtype of '288' is changed from 'float32' to 'int8' in rknn model for performance!
                      Please take care of this change when deploy rknn model with Runtime API!
I rknn building ...
I rknn building done.


done
--> Export rknn model
done
